In [1]:
from coding import search_w

import numpy as np
from scipy import fftpack as FT

# Let's debug Cyclic Code a bit

## n = 7, s = 2, d = 10

In [2]:
# global variables:
n = 7
s = 2
d = 10
#d = 1

In [3]:
np.set_printoptions(precision=4,linewidth=200.0)
W, fake_W, W_perp, S, C_1 = search_w(n, s)

C_1 is: 
[[ 0.3780+0.j      0.3780+0.j      0.3780+0.j    ]
 [ 0.3780+0.j      0.2357-0.2955j -0.0841-0.3685j]
 [ 0.3780+0.j     -0.0841-0.3685j -0.3405+0.164j ]
 [ 0.3780+0.j     -0.3405-0.164j   0.2357+0.2955j]
 [ 0.3780+0.j     -0.3405+0.164j   0.2357-0.2955j]
 [ 0.3780+0.j     -0.0841+0.3685j -0.3405-0.164j ]
 [ 0.3780+0.j      0.2357+0.2955j -0.0841+0.3685j]]

Q is: 
[[ 1.0000 +0.0000e+00j  1.0000 +0.0000e+00j  1.0000 +0.0000e+00j  1.0000 +0.0000e+00j  1.0000 +0.0000e+00j  1.0000 +0.0000e+00j  1.0000 +0.0000e+00j]
 [-0.4010 -1.7568e+00j  1.1235 -1.4088e+00j  1.8019 -3.3307e-16j  1.1235 +1.4088e+00j -0.4010 +1.7568e+00j -1.6235 +7.8183e-01j -1.6235 -7.8183e-01j]
 [-0.9010 +4.3388e-01j -0.2225 -9.7493e-01j  1.0000 +0.0000e+00j -0.2225 +9.7493e-01j -0.9010 -4.3388e-01j  0.6235 -7.8183e-01j  0.6235 +7.8183e-01j]]

W is: 
[[ -1.1412e-01 -5.0000e-01j   7.1850e-01 -9.0097e-01j   1.4370e+00 -1.2589e-16j   7.1850e-01 +9.0097e-01j  -1.1412e-01 +5.0000e-01j  -4.1124e-16 +2.6060e-16j  -4.1058

In [4]:
def estimator_generator(n, s):
    estimator = np.zeros((n, s+1), dtype=complex)
    #z_gen_func = np.vectorize(lambda t: np.exp(-2*np.pi*t*1j/n))
    z_gen_func = np.vectorize(lambda t: np.exp(2*np.pi*t*1j/n))
    col1 = z_gen_func(np.arange(n))
    for i in range(s+1):
        estimator[:, i] = np.power(col1, i)
    return estimator

p_estimator = estimator_generator(n, s)
print(p_estimator)

[[ 1.0000+0.j      1.0000+0.j      1.0000+0.j    ]
 [ 1.0000+0.j      0.6235+0.7818j -0.2225+0.9749j]
 [ 1.0000+0.j     -0.2225+0.9749j -0.9010-0.4339j]
 [ 1.0000+0.j     -0.9010+0.4339j  0.6235-0.7818j]
 [ 1.0000+0.j     -0.9010-0.4339j  0.6235+0.7818j]
 [ 1.0000+0.j     -0.2225-0.9749j -0.9010+0.4339j]
 [ 1.0000+0.j      0.6235-0.7818j -0.2225-0.9749j]]


## simulate epsilon here (assume worker 0 1 fails):

In [5]:
simulated_epsilon = np.zeros((n, d), dtype=complex)
simulated_epsilon[0:2,:] = -100.0+0.0j
print(simulated_epsilon)

[[-100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j]
 [-100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]]


## obtain E_2 (simulated) here:

In [6]:
simulated_E2 = np.dot(W_perp, simulated_epsilon)
print(simulated_E2)

[[ -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j]
 [ -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j]
 [-29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j]
 [-61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j]]


## calculated a to get the full E matrix

In [8]:
import copy
poly_a = np.zeros(s+1, dtype=complex)
poly_a[-1] = 1+0j
fake_poly_a = copy.deepcopy(poly_a)
# calculate a over here:
E_2 = simulated_E2

def _cls_solver(A, b):
    return np.dot(np.dot(np.linalg.inv(np.dot(_array_getH(A), A)), _array_getH(A)),b)

def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

_shape = E_2.shape
_s = _shape[0]/2
_d = _shape[1]
_X = np.take(E_2, np.array([range(-i-(_s+1), -i-2+1) for i in range(_s)]).reshape(-1), axis=0).reshape((_s, _s*_d), order='F')
# we use tmp_y as the start point to obtain the full E matrix
tmp_y = np.take(E_2, np.array([-i-1 for i in range(_s)]), axis=0)
_y = tmp_y.reshape(-1, order='F')
alpha_normal = _cls_solver(np.transpose(_X), _y.reshape(_y.shape[0], 1))
poly_a[0:s] = alpha_normal.reshape(-1)
fake_poly_a[0:s] = -alpha_normal.reshape(-1)

# check matrix formula and calculated result:
print("Formulated X is :")
print(np.transpose(_X))
print
print("Formulated y is :")
print(_y.reshape(_y.shape[0], 1))
print
print("E_2 is: ")
print(E_2)
print
print("Polynomial a is: ")
print(poly_a)

Formulated X is :
[[ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]]

Formulated y is :
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.

In [9]:
_X_v2 = np.take(E_2, np.array([range(-i-(_s+1), -i-2+1) for i in range(_s)]).reshape(-1), axis=0)

random_factor = np.random.normal(size=_d)
#random_factor = np.ones(_d)
print("One dim of X_v2 {}".format(_X_v2[:, 0]))
print
aaa = np.dot(_X_v2, random_factor)
print("=======================================================")
print(_X_v2)
print(random_factor)
print("=======================================================")
#_X_v2_4solve = aaa.reshape((_s, _s), order='F')
_X_v2_4solve = aaa.reshape((_s, _s))
print
print("Combined X is: ")
print(_X_v2_4solve)
tmp_y = np.take(E_2, np.array([-i-1 for i in range(_s)]), axis=0)

bbb = np.dot(tmp_y, random_factor)
alpha = _cls_solver(_X_v2_4solve, bbb.reshape(bbb.shape[0], 1))
print
print("solved a is: {}".format(alpha))
print(alpha_normal)
print("########################################################")
print
print("new alpha (combined): ")
print(np.dot(_X_v2_4solve, alpha))
print
print("old alpha (not combined): ")
print(np.dot(_X_v2_4solve, alpha_normal))

One dim of X_v2 [ -3.7430+16.3993j -29.3859+36.8488j  -3.7430-16.3993j  -3.7430+16.3993j]

[[ -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j]
 [-29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j]
 [ -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j]]
[-0.3792  0.1752 -0.1907  0.8075  0.9145 -0.2917  1.1999  0.7037  0.7541 -0.8684]

Combined X is: 
[[-10.5740 +46.3278j -83.0152+104.0977j]
 [-10.5740 -46.3278j -10.5740 +46.32

In [10]:
# function to obtain matrix E
def obtain_E(alpha, E_2, s):
    # obtain E_1 in shape of n-2s by d
    _processing_y = np.transpose(E_2)[:, -s:]

    for i in range(n-2*s):
        tmp = np.dot(_processing_y[:,-s:], alpha)
        print("Obtaining E[{}]".format(i))
        print(_processing_y[:,-s:])
        print
        print("Calculated Results:")
        print(tmp)
        print
        _processing_y = np.concatenate((_processing_y, tmp), axis=1)
        #print(_processing_y[0,-s:])
        #print('============================================================')
    #print(_processing_y[0, s:])
    #print("----------------------------------------------------------------")
    #exit()
    return np.transpose(_processing_y[:, s:])

# function to obtain epsilon
def obtain_epsilon(E):
    return FT.ifft(E, n=n, axis=0)

## d=10 has too many dims? Let's look just one dimension then

In [11]:
# if we look only one dimension: i.e. d=1
# check matrix formula and calculated result:
print("Formulated X (d=1) is :")
print(np.transpose(_X)[0:2, :])
print
print("Formulated y is (d=1) :")
print(_y.reshape(_y.shape[0], 1)[0:2])
print
print("E_2 is: ")
print(E_2[:, 0])
print
print("X * alpha, shoud equals to y")
print(np.dot(np.transpose(_X)[0:2, :],alpha))

Formulated X (d=1) is :
[[ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]]

Formulated y is (d=1) :
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]]

E_2 is: 
[ -3.7430-16.3993j  -3.7430+16.3993j -29.3859+36.8488j -61.3621+29.5505j]

X * alpha, shoud equals to y
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]]


# Test New Method

In [12]:
q1 = np.exp(-2*np.pi*0*1j/n)
q2 = np.exp(-2*np.pi*1*1j/n)
q1_conj = np.conj(q1)
q2_conj = np.conj(q2)
print("q_1 is {}".format(q1))
print("q_2 is {}".format(q2))
print("q_1_conj is {}".format(q1_conj))
print("q_2_conj is {}".format(q2_conj))
#bbb = - (q1 + q2)
#ccc = q1 * q2
bbb = - (q1_conj + q2_conj)
ccc = q1_conj * q2_conj
print(bbb)
print(ccc)
print
real_poly=[ccc, bbb, 1]
print("The Correct polynomial should be: {}".format(real_poly))
print("The one we get is: {}".format(poly_a))
print
print(np.dot(p_estimator, real_poly))

q_1 is (1+0j)
q_2 is (0.623489801859-0.781831482468j)
q_1_conj is (1-0j)
q_2_conj is (0.623489801859+0.781831482468j)
(-1.62348980186-0.781831482468j)
(0.623489801859+0.781831482468j)

The Correct polynomial should be: [(0.62348980185873359+0.7818314824680298j), (-1.6234898018587336-0.7818314824680298j), 1]
The one we get is: [-0.6235-0.7818j  1.6235+0.7818j  1.0000+0.j    ]

[ 0.0000 +0.0000e+00j  0.0000 +1.1102e-16j  0.8460 -1.0609e+00j  3.0489 -4.4409e-16j  2.3705 +2.9725e+00j -0.6784 +2.9725e+00j -1.2225 +5.8874e-01j]


In [38]:
_row_vec = np.zeros((1, n-2*s))
_row_vec[0][-1]=1
_recover_final=np.zeros((1,n),dtype=complex)
#print(np.dot(p_estimator, poly_a))
# if we let a = -a
#print
#print(np.dot(p_estimator, fake_poly_a))
print("="*80)
estimation = np.dot(p_estimator, fake_poly_a)
print("Step 0: The results we test the zeros points: ")
print(estimation)
print("="*80)
print
err_indices = [i for i, elem in enumerate(estimation) if (elem.real > 1e-10 or elem.imag > 1e-10)]
print("="*80)
print("Step 1: C_1 matrix we have: ")
print(C_1)
print("="*80)
print
recover=C_1.take(err_indices, axis=0).take(np.arange(n-2*s),axis=0)
print("="*80)
print("Step 2: (n-2s)by(n-2s) matrix we get: ")
print(recover)
print("="*80)
print
remaining_indices = err_indices[0:n-2*s]

inv_recover=np.linalg.inv(recover)
print("="*80)
print("Step 3: inv of the (n-2s)by(n-2s) matrix: ")
print(inv_recover)
print("check result: ")
print(np.dot(inv_recover, recover))
print("="*80)
print

row_recover = np.dot(_row_vec, inv_recover)
print("="*80)
print("Step 4: [0, ..., 1] vec we use: ")
print(_row_vec)
print("dot product it with inv :")
print(row_recover)
print("="*80)
print


_recover_final[0][[remaining_indices]] = row_recover[0]
print("="*80)
print("Step 5: insert zeros to appropriate positions ")
print(_recover_final)
print("="*80)
print
#decoded_grad = np.dot(_recover_final, R)

Step 0: The results we test the zeros points: 
[ -1.3323e-15 -1.4433e-15j  -1.1657e-15 -1.2212e-15j   8.4601e-01 -1.0609e+00j   3.0489e+00 -1.5543e-15j   2.3705e+00 +2.9725e+00j  -6.7845e-01 +2.9725e+00j  -1.2225e+00 +5.8874e-01j]

Step 1: C_1 matrix we have: 
[[ 0.3780+0.j      0.3780+0.j      0.3780+0.j    ]
 [ 0.3780+0.j      0.2357-0.2955j -0.0841-0.3685j]
 [ 0.3780+0.j     -0.0841-0.3685j -0.3405+0.164j ]
 [ 0.3780+0.j     -0.3405-0.164j   0.2357+0.2955j]
 [ 0.3780+0.j     -0.3405+0.164j   0.2357-0.2955j]
 [ 0.3780+0.j     -0.0841+0.3685j -0.3405-0.164j ]
 [ 0.3780+0.j      0.2357+0.2955j -0.0841+0.3685j]]

Step 2: (n-2s)by(n-2s) matrix we get: 
[[ 0.3780+0.j     -0.0841-0.3685j -0.3405+0.164j ]
 [ 0.3780+0.j     -0.3405-0.164j   0.2357+0.2955j]
 [ 0.3780+0.j     -0.3405+0.164j   0.2357-0.2955j]]

Step 3: inv of the (n-2s)by(n-2s) matrix: 
[[-0.4339 +1.9010e+00j  3.5135 -1.1749e-15j -0.4339 -1.9010e+00j]
 [-0.7818 +3.4254e+00j  3.9474 -1.9010e+00j -3.1656 -1.5245e+00j]
 [-0.4339 +

True 1 traspose * G: <br />
[[ 0.0006 +0.0000e+00j  0.0006 +0.0000e+00j  0.0006 +0.0000e+00j ...,  0.0007 +6.7763e-21j  0.0007 +6.7763e-21j  0.0007 +6.7763e-21j]]

The results we get:  <br />
[ -2.8031e-04 -1.8633e-05j  -2.8031e-04 -1.8633e-05j  -2.8031e-04 -1.8633e-05j ...,  -4.5684e-05 -1.5724e-04j  -4.5684e-05 -1.5724e-04j  -4.5684e-05 -1.5724e-04j]

## Full E matrix calculated

In [26]:
# now let's find the whole E and also epsilon
E_1=obtain_E(alpha, E_2, s)
# concatenate E_1 and E_2 to obtain E
E = np.concatenate((E_1, E_2), axis=0)

epsilon = obtain_epsilon(E)

#print(epsilon)

Obtaining E[0]
[[-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]]

Calculated Results:
[[-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]]

Obtaining E[1]
[[-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j 

## finally let's look at the epsilon matrix we get

In [27]:
print(epsilon)

[[ -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j
   -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j]
 [  4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j
    4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j]
 [  3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j
    3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j]
 [ -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j
   -5.1483e-14 +2.080

## we if we give it a factor? say sqrt(n)

In [28]:
print(np.sqrt(n)*epsilon)

[[ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j]
 [ -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j
   -1.3621e-13 +5.503

In [29]:
# let's try to fix it
tmp = np.sqrt(n)*epsilon
a = np.roll(tmp, s-1, axis=0)
print(a)

[[ -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j
   -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j]
 [ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.182

# estimation as s grows

In [4]:
print("s=1")

s=1


[  2.2204e-16 -8.5158e-16j  -9.7319e-03 +1.3917e-01j  -3.8738e-02 +2.7564e-01j  -8.6455e-02 +4.0674e-01j  -1.5195e-01 +5.2992e-01j  -2.3396e-01 +6.4279e-01j  -3.3087e-01 +7.4314e-01j
  -4.4081e-01 +8.2904e-01j  -5.6163e-01 +8.9879e-01j  -6.9098e-01 +9.5106e-01j  -8.2635e-01 +9.8481e-01j  -9.6510e-01 +9.9939e-01j  -1.1045e+00 +9.9452e-01j  -1.2419e+00 +9.7030e-01j
  -1.3746e+00 +9.2718e-01j  -1.5000e+00 +8.6603e-01j  -1.6157e+00 +7.8801e-01j  -1.7193e+00 +6.9466e-01j  -1.8090e+00 +5.8779e-01j  -1.8829e+00 +4.6947e-01j  -1.9397e+00 +3.4202e-01j
  -1.9781e+00 +2.0791e-01j  -1.9976e+00 +6.9756e-02j  -1.9976e+00 -6.9756e-02j  -1.9781e+00 -2.0791e-01j  -1.9397e+00 -3.4202e-01j  -1.8829e+00 -4.6947e-01j  -1.8090e+00 -5.8779e-01j
  -1.7193e+00 -6.9466e-01j  -1.6157e+00 -7.8801e-01j  -1.5000e+00 -8.6603e-01j  -1.3746e+00 -9.2718e-01j  -1.2419e+00 -9.7030e-01j  -1.1045e+00 -9.9452e-01j  -9.6510e-01 -9.9939e-01j
  -8.2635e-01 -9.8481e-01j  -6.9098e-01 -9.5106e-01j  -5.6163e-01 -8.9879e-01j  -4.4081e-01 -8.2904e-01j  -3.3087e-01 -7.4314e-01j  -2.3396e-01 -6.4279e-01j  -1.5195e-01 -5.2992e-01j
  -8.6455e-02 -4.0674e-01j  -3.8738e-02 -2.7564e-01j  -9.7319e-03 -1.3917e-01j]

In [5]:
print("s=3")

s=3


[ -1.5844e-07 -3.1324e-07j  -1.6520e-07 +6.8121e-07j   2.8651e-07 -2.0750e-07j   1.2002e-02 -1.0808e-02j   5.5263e-02 -3.1909e-02j   1.4914e-01 -4.8465e-02j   3.0509e-01 -3.2078e-02j
   5.1964e-01 +5.4598e-02j   7.6993e-01 +2.5014e-01j   1.0132e+00 +5.8491e-01j   1.1904e+00 +1.0718e+00j   1.2342e+00 +1.6987e+00j   1.0798e+00 +2.4251e+00j   6.7673e-01 +3.1833e+00j
   1.1160e-04 +3.8840e+00j  -9.4092e-01 +4.4273e+00j  -2.0996e+00 +4.7162e+00j  -3.3935e+00 +4.6711e+00j  -4.7118e+00 +4.2428e+00j  -5.9278e+00 +3.4226e+00j  -6.9144e+00 +2.2468e+00j
  -7.5609e+00 +7.9488e-01j  -7.7876e+00 -8.1831e-01j  -7.5576e+00 -2.4554e+00j  -6.8819e+00 -3.9730e+00j  -5.8193e+00 -5.2395e+00j  -4.4688e+00 -6.1504e+00j  -2.9572e+00 -6.6416e+00j
  -1.4233e+00 -6.6952e+00j  -1.4793e-04 -6.3404e+00j   1.2002e+00 -5.6474e+00j   2.0996e+00 -4.7161e+00j   2.6603e+00 -3.6617e+00j   2.8862e+00 -2.5988e+00j   2.8183e+00 -1.6272e+00j
   2.5246e+00 -8.2035e-01j   2.0881e+00 -2.1951e-01j   1.5929e+00 +1.6739e-01j   1.1125e+00 +3.6147e-01j   7.0103e-01 +4.0472e-01j   3.8825e-01 +3.4958e-01j   1.8030e-01 +2.4815e-01j
   6.3773e-02 +1.4323e-01j   1.3264e-02 +6.2404e-02j  -5.0659e-07 +1.6144e-02j]

In [6]:
print("s=5")

s=5


[  4.8147e-03 +3.3925e-03j   4.3710e-03 -1.2087e-02j  -4.0984e-03 -6.1409e-03j  -1.2313e-02 -3.1390e-03j  -1.4381e-04 +6.5030e-03j   2.4387e-02 +5.7570e-02j   8.8384e-03 +1.6961e-01j
  -1.1812e-01 +3.0418e-01j  -3.8891e-01 +3.5480e-01j  -7.4083e-01 +1.8814e-01j  -1.0020e+00 -2.7474e-01j  -9.4285e-01 -9.7665e-01j  -3.8346e-01 -1.6916e+00j   6.8816e-01 -2.0744e+00j
   2.0465e+00 -1.7885e+00j   3.2556e+00 -6.6719e-01j   3.8023e+00 +1.1632e+00j   3.2936e+00 +3.2630e+00j   1.6429e+00 +4.9897e+00j  -8.3924e-01 +5.7125e+00j  -3.5017e+00 +5.0545e+00j
  -5.5503e+00 +3.0650e+00j  -6.3220e+00 +2.4574e-01j  -5.5319e+00 -2.5925e+00j  -3.3954e+00 -4.5972e+00j  -5.7154e-01 -5.1698e+00j   2.0562e+00 -4.1762e+00j   3.6831e+00 -1.9930e+00j
   3.8560e+00 +6.3050e-01j   2.6149e+00 +2.8391e+00j   4.5448e-01 +3.9668e+00j  -1.8729e+00 +3.7485e+00j  -3.6323e+00 +2.3771e+00j  -4.3518e+00 +3.9219e-01j  -3.9508e+00 -1.5408e+00j
  -2.7147e+00 -2.8721e+00j  -1.1428e+00 -3.3356e+00j   2.5831e-01 -2.9896e+00j   1.1492e+00 -2.1342e+00j   1.4388e+00 -1.1543e+00j   1.2557e+00 -3.6368e-01j   8.4215e-01 +8.8841e-02j
   4.2998e-01 +2.2505e-01j   1.5366e-01 +1.7170e-01j   3.1291e-02 +6.9218e-02j]

In [7]:
print("s=15")

s=15


[ -2.3192e-03 +9.8397e-02j  -2.9488e-01 +6.6184e-01j  -2.0381e+00 +1.7679e+00j  -6.5749e+00 +1.5774e+00j  -1.2323e+01 -3.5575e+00j  -1.3281e+01 -1.4409e+01j  -4.5088e+00 -2.4531e+01j
   1.0192e+01 -2.4892e+01j   2.0185e+01 -1.4245e+01j   1.9194e+01 -1.1560e+00j   1.1120e+01 +5.6433e+00j   3.7224e+00 +5.3423e+00j   4.2086e-01 +2.5336e+00j  -1.5049e-01 +6.6115e-01j
  -4.7308e-02 +7.6127e-02j   2.0796e-01 -7.3961e-01j   5.0748e+00 -3.3090e+00j   2.3420e+01 +4.5488e+00j   3.3435e+01 +5.2870e+01j  -3.4850e+01 +1.1693e+02j  -1.7176e+02 +7.1403e+01j
  -1.9462e+02 -1.1327e+02j  -1.6175e+01 -2.1324e+02j   1.2792e+02 -8.2978e+01j   5.5859e+01 +6.0877e+01j  -5.5588e+01 +3.9152e+01j  -7.3624e+01 -1.7366e+01j  -8.8155e+01 -5.8991e+01j
  -4.1784e+01 -1.4246e+02j   9.2246e+01 -8.9413e+01j   3.1309e+00 +9.6531e+01j  -2.8172e+02 -4.0604e+01j  -1.5550e+02 -4.7537e+02j   4.0311e+02 -4.6946e+02j   5.8104e+02 +9.0140e+01j
   1.5576e+02 +4.0096e+02j  -1.6506e+02 +1.5453e+02j  -4.0626e+01 -8.7849e+01j   1.2148e+02 -8.2534e+00j   5.9357e+01 +1.1543e+02j  -4.6183e+01 +8.4386e+01j  -5.1664e+01 +8.8689e+00j
  -1.6066e+01 -1.2906e+01j  -5.1347e-01 -5.2808e+00j   4.0988e-01 -5.3854e-01j]